NLP project

In [2]:
import sys  
sys.path.insert(1, './transformer')

In [3]:
from dataset import *
from Transformer import *
from utils import *

In [4]:
input_path = './data/gec-only/train.src.tok'
target_path = './data/gec-only/train.tgt.tok'

dataset = construct_dataset(input_path, target_path, 856*2, 16)

In [5]:
len(dataset)

2047

In [6]:
for (batch, (input_ids, target_ids)) in enumerate(dataset):
    print(batch)
    print(input_ids)
    print(target_ids)
    break

0
tf.Tensor(
[[    0 10431 47489 ...     1     1     1]
 [    0 47447    13 ...     1     1     1]
 [    0 25417 17772 ...     1     1     1]
 ...
 [    0 36765 25482 ...     1     1     1]
 [    0 36709 34251 ...     1     1     1]
 [    0 25417 10965 ...     1     1     1]], shape=(16, 1711), dtype=int32)
tf.Tensor(
[[10431 47489 50118 ...     1     1     1]
 [47447    13  1470 ...     1     1     1]
 [25417 17772 36709 ...     1     1     1]
 ...
 [36765 25482 36709 ...     1     1     1]
 [36709 34251 22063 ...     1     1     1]
 [25417 10965 41171 ...     1     1     1]], shape=(16, 1711), dtype=int32)


In [7]:
# Initialize the Transformer model
num_layers = 6
d_model = 512
num_heads = 8
dff = 2048
target_vocab_size = len(dataset)
pe_target = 1711
rate = 0.01
target_vocab_size = len(target_ids)
transformer_model = Transformer(num_layers, d_model, num_heads, dff,
                                target_vocab_size, pe_target, rate)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [9]:
# Define loss function, optimizer, and other necessary components (as shown in the previous response)

# Training step function
@tf.function
def train_step(input_ids, target_ids):
    tar_inp = target_ids[:, :-1]
    tar_real = target_ids[:, 1:]

    # Create masks
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input_ids, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer_model(input_ids, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer_model.trainable_variables))

    return loss

In [13]:
# Training loop
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    for (batch, (input_ids, target_ids)) in enumerate(dataset):  # Assuming dataset is prepared
        batch_loss = train_step(input_ids, target_ids)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    print(f'Epoch {epoch + 1} Loss {total_loss / (batch + 1):.4f}')

KeyError: in user code:

    File "C:\Users\darit\AppData\Local\Temp\ipykernel_12844\3181115009.py", line 17, in train_step  *
        optimizer.apply_gradients(zip(gradients, transformer_model.trainable_variables))
    File "C:\Studying\nlp\nlp-kma-gec-project\venv\lib\site-packages\keras\src\optimizers\optimizer.py", line 1230, in apply_gradients  **
        return super().apply_gradients(grads_and_vars, name=name)
    File "C:\Studying\nlp\nlp-kma-gec-project\venv\lib\site-packages\keras\src\optimizers\optimizer.py", line 652, in apply_gradients
        iteration = self._internal_apply_gradients(grads_and_vars)
    File "C:\Studying\nlp\nlp-kma-gec-project\venv\lib\site-packages\keras\src\optimizers\optimizer.py", line 1260, in _internal_apply_gradients
        return tf.__internal__.distribute.interim.maybe_merge_call(
    File "C:\Studying\nlp\nlp-kma-gec-project\venv\lib\site-packages\keras\src\optimizers\optimizer.py", line 1352, in _distributed_apply_gradients_fn
        distri